In [1]:
import gym
from collections import namedtuple
import numpy as np
#from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim


In [2]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70


In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)


In [4]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


In [5]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs



In [6]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean



In [7]:
env = gym.make("CartPole-v0")
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
# writer = SummaryWriter(comment="-cartpole")

for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    # writer.add_scalar("loss", loss_v.item(), iter_no)
    # writer.add_scalar("reward_bound", reward_b, iter_no)
    # writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break


0: loss=0.682, reward_mean=31.9, reward_bound=34.5
1: loss=0.670, reward_mean=38.1, reward_bound=45.5
2: loss=0.658, reward_mean=30.8, reward_bound=33.5
3: loss=0.646, reward_mean=32.1, reward_bound=32.5
4: loss=0.649, reward_mean=36.4, reward_bound=42.0
5: loss=0.631, reward_mean=43.0, reward_bound=45.0
6: loss=0.615, reward_mean=43.2, reward_bound=49.5
7: loss=0.606, reward_mean=52.4, reward_bound=63.0
8: loss=0.606, reward_mean=66.8, reward_bound=78.5
9: loss=0.577, reward_mean=51.0, reward_bound=52.0
10: loss=0.581, reward_mean=45.2, reward_bound=46.0
11: loss=0.593, reward_mean=58.8, reward_bound=63.5
12: loss=0.575, reward_mean=53.7, reward_bound=67.5
13: loss=0.554, reward_mean=53.7, reward_bound=60.5
14: loss=0.569, reward_mean=60.8, reward_bound=76.5
15: loss=0.556, reward_mean=66.0, reward_bound=73.0
16: loss=0.541, reward_mean=64.2, reward_bound=65.5
17: loss=0.549, reward_mean=57.8, reward_bound=59.0
18: loss=0.536, reward_mean=64.8, reward_bound=62.5
19: loss=0.537, reward